<a href="https://colab.research.google.com/github/Bearbrice/maDLC_MM_hands_trained/blob/main/Berclaz_Colab_maDLC_Analyze.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hands tracking with DeepLabCut - Training and evaluation of the network

> Using multi-animal DLC (maDLC)

This notebook illustrates how to, for multi-animal projects, use the cloud-based GPU to:
- evaluate a network
- cross evaluate inference parameters
- analyze novel videos
- assemble tracklets
- create quality check plots

## Credits and references:
* Cloned from https://github.com/AlexEMG/DeepLabCut

* Nath\*, Mathis\* et al.: Using DeepLabCut for markerless pose estimation during behavior across species. Nature Protocols, 2019.

* Paper: https://www.nature.com/articles/s41596-019-0176-0

* Pre-print: https://www.biorxiv.org/content/biorxiv/early/2018/11/24/476531.full.pdf

* Adaptation: Brice Berclaz






## First, go to "Runtime" ->"change runtime type"->select "Python3", and then select "GPU"


In [1]:
#(this will take a few minutes to install all the dependences!)
!pip install deeplabcut
%reload_ext numpy
%reload_ext scipy
%reload_ext matplotlib
%reload_ext mpl_toolkits

     |████████████████████████████████| 696kB 8.0MB/s 
     |████████████████████████████████| 13.1MB 229kB/s 
     |████████████████████████████████| 29.2MB 104kB/s 
     |████████████████████████████████| 20.0MB 1.4MB/s 
     |████████████████████████████████| 112kB 54.3MB/s 
     |████████████████████████████████| 9.5MB 41.7MB/s 
     |████████████████████████████████| 21.6MB 1.6MB/s 
     |████████████████████████████████| 184kB 55.2MB/s 
     |████████████████████████████████| 3.1MB 35.8MB/s 
     |████████████████████████████████| 296kB 56.1MB/s 
     |████████████████████████████████| 552kB 40.8MB/s 
  Created wheel for filterpy: filename=filterpy-1.4.5-cp37-none-any.whl size=110476 sha256=1c3b5816addb25dad6a5b23aad1e3087d00d7176840ac61d39d80855e60d4198
  Stored in directory: /root/.cache/pip/wheels/c3/0c/dd/e92392c3f38a41371602d99fc77d6c1d42aadbf0c6afccdd02
  Created wheel for bayesian-optimization: filename=bayesian_optimization-1.2.0-cp37-none-any.whl size=11686 sha256=87fa1e

**(Be sure to click "RESTART RUNTIME" is it is displayed above above before moving on !)**

Set up DLC!

In [1]:
# Use TensorFlow 1.x:
%tensorflow_version 1.x

#GUIs don't work on the cloud, so label your data locally on your computer! This will suppress the GUI support
import os
os.environ["DLClight"]="True"
import deeplabcut
deeplabcut.__version__

TensorFlow 1.x selected.
DLC loaded in light mode; you cannot use any GUI (labeling, relabeling and standalone GUI)


'2.1.10.4'

## Get the data from repository

In [3]:
# Clone the github repository
!git clone -l -s git://github.com/Bearbrice/maDLC_MM_hands_trained.git
#cloned-DLC_MM_hands
#%cd cloned-DLC_MM_hands
!ls

Cloning into 'maDLC_MM_hands_trained'...
remote: Enumerating objects: 192, done.
remote: Counting objects: 100% (192/192), done.
remote: Compressing objects: 100% (175/175), done.
remote: Total 192 (delta 19), reused 180 (delta 14), pack-reused 0
Receiving objects: 100% (192/192), 133.67 MiB | 8.18 MiB/s, done.
Resolving deltas: 100% (19/19), done.
maDLC_MM_hands_trained	sample_data


In [17]:
#create a path variable that links to the config file:
path_config_file = '/content/maDLC_MM_hands_trained/maDLC_MM_hands-Brice-2021-07-08/config.yaml'
print(path_config_file)


VideoType = 'mp4' #, mp4, MOV, or avi, whatever you uploaded!


# No need to edit this, we are going to assume you put videos you want to analyze in the "videos" folder, but if this is NOT true, edit below:
videofile_path = ['/content/maDLC_MM_hands_trained/Inference_videos'] #Enter the list of videos or folder to analyze.
print(videofile_path)

# Loading example data set:
# deeplabcut.load_demo_data(path_config_file)
shuffle=1

/content/maDLC_MM_hands_trained/maDLC_MM_hands-Brice-2021-07-08/config.yaml
['/content/maDLC_MM_hands_trained/Inference_videos']


## Start evaluating: for maDLC, this is several steps. 
 - First, we evaluate the pose estimation performance, and then we can cross-valudate optimal inference parameters.

- This funtion evaluates a trained model for a specific shuffle/shuffles at a particular state or all the states on the data set (images) and stores the results as .5 and .csv file in a subdirectory under **evaluation-results**

- If the scoremaps do not look accurate, don't proceed to tracklet assembly; please consider (1) adding more data, (2) adding more bodyparts!

Here is an example of what you'd aim to see before proceeding:

![alt text](https://images.squarespace-cdn.com/content/v1/57f6d51c9f74566f55ecf271/1590535809087-X655WY9W1MW1MY1I7DHE/ke17ZwdGBToddI8pDm48kBoswZhKnUtAF7-bTXgw67EUqsxRUqqbr1mOJYKfIPR7LoDQ9mXPOjoJoqy81S2I8N_N4V1vUb5AoIIIbLZhVYxCRW4BPu10St3TBAUQYVKc5tTP1cnANTUwNNPnYFjIp6XbP9N1GxIgAkxvBVqt0UvLpPHYwvNQTwHg8f_Zu8ZF/evaluation.png?format=1000w)

In [7]:
#let's evaluate first:
deeplabcut.evaluate_network(path_config_file,Shuffles=[shuffle], plotting=True,c_engine=False)
#plot a few scoremaps:
deeplabcut.extract_save_all_maps(path_config_file, shuffle=shuffle, Indices=[0])

#%matplotlib notebook
#deeplabcut.evaluate_network(path_config_file,plotting=True)

# Here you want to see a low pixel error! Of course, it can only be as good as the labeler, 
#so be sure your labels are good! (And you have trained enough ;)

Running  DLC_resnet50_maDLC_MM_handsJul8shuffle1_10000  with # of trainingiterations: 10000
Initializing ResNet
Activating extracting of PAFs


0it [00:00, ?it/s]

Analyzing data...


1it [00:08,  8.08s/it]/usr/local/lib/python3.7/dist-packages/deeplabcut/pose_estimation_tensorflow/evaluate_multianimal.py:330: RuntimeWarning: All-NaN slice encountered
  distnorm[imageindex] = np.nanmax(lengths)
2it [00:08,  5.75s/it]/usr/local/lib/python3.7/dist-packages/deeplabcut/pose_estimation_tensorflow/evaluate_multianimal.py:330: RuntimeWarning: All-NaN slice encountered
  distnorm[imageindex] = np.nanmax(lengths)
3it [00:08,  4.10s/it]/usr/local/lib/python3.7/dist-packages/deeplabcut/pose_estimation_tensorflow/evaluate_multianimal.py:330: RuntimeWarning: All-NaN slice encountered
  distnorm[imageindex] = np.nanmax(lengths)
4it [00:08,  2.95s/it]/usr/local/lib/python3.7/dist-packages/deeplabcut/pose_estimation_tensorflow/evaluate_multianimal.py:330: RuntimeWarning: All-NaN slice encountered
  distnorm[imageindex] = np.nanmax(lengths)
5it [00:09,  2.20s/it]/usr/local/lib/python3.7/dist-packages/deeplabcut/pose_estimation_tensorflow/evaluate_multianimal.py:330: RuntimeWarning: 

Results for 10000 training iterations: 95, shuffle 1:
Train error: 2.49 pixels. Test error: 4.73 pixels.
With pcutoff of 0.6:
Train error: 2.49 pixels. Test error: 3.87 pixels.
##########################################
Average Euclidean distance to GT per individual (in pixels)
individuals
leftHand     2.661212
rightHand    2.507637
Average Euclidean distance to GT per bodypart (in pixels)
bodyparts
indexEnd       2.513913
indexStart     2.796565
littleEnd      2.514180
littleStart    3.152753
middleEnd      2.512030
middleStart    2.475623
ringEnd        2.310887
ringStart      2.605412
thumbEnd       2.065746
thumbStart     2.815365
Done and results stored for snapshot:  snapshot-10000
/content/maDLC_MM_hands_trained/maDLC_MM_hands-Brice-2021-07-08/evaluation-results/  already exists!
/content/maDLC_MM_hands_trained/maDLC_MM_hands-Brice-2021-07-08/evaluation-results/iteration-0/maDLC_MM_handsJul8-trainset95shuffle1  already exists!
Initializing ResNet
Activating extracting of PAFs


0it [00:00, ?it/s]

Analyzing data...


1it [00:00,  2.53it/s]
  0%|          | 0/1 [00:00<?, ?it/s]

Saving plots...


100%|██████████| 1/1 [00:02<00:00,  2.12s/it]


IF these images, numbers, and maps do not look good, do not proceed. You should increase the diversity and number of frames you label, and re-create a training dataset and re-train! 

NOTE: this optimized part detection for video analysis. It cannot optimze for tracking, as this is use-case dependent. Please check the docs on how you can set the best parameters and modify/test before "final" tracking parameters. You can use COLAB to analyze videos, but afterwards we recommend using the outputs/proejct folder locally to run the final steps! They do not require a GPU. 

In [8]:
#Cross-validate for Animal Assembly & Tracking:
deeplabcut.evaluate_multianimal_crossvalidate(
            path_config_file,
            Shuffles=[shuffle],
            edgewisecondition=True,
            leastbpts=1,
            init_points=20,
            n_iter=100,
            target='rpck_train',
        )

/content/maDLC_MM_hands_trained/maDLC_MM_hands-Brice-2021-07-08/evaluation-results/iteration-0/maDLC_MM_handsJul8-trainset95shuffle1  already exists!
/content/maDLC_MM_hands_trained/maDLC_MM_hands-Brice-2021-07-08/evaluation-results/iteration-0/maDLC_MM_handsJul8-trainset95shuffle1/DLC_resnet50_maDLC_MM_handsJul8shuffle1_10000-snapshot-10000.h5
Computing distances...
rpck 0.37520657777041505 rpck train: 0.3938144397370296
rmse 3.6271651701913545 miss 12.0 hit 8.0
rpck 0.864166138291354 rpck train: 0.9603479170971787
rmse 4.70389295490481 miss 1.0 hit 19.0
rpck 0.4695619176837278 rpck train: 0.6350334422920736
rmse 4.274095522388612 miss 9.666666666666666 hit 10.333333333333334
rpck 0.12670527247403626 rpck train: 0.169594979252035
rmse 53.75273011626896 miss 16.4 hit 3.6
rpck 0.864166138291354 rpck train: 0.9562741869537633
rmse 4.70389295490481 miss 1.0 hit 19.0
rpck 0.7178504501167947 rpck train: 0.7657251486056164
rmse 6.7722762466710416 miss 4.166666666666667 hit 15.833333333333334

## Start Analyzing videos: 
This function analyzes the new video. The user can choose the best model from the evaluation results and specify the correct snapshot index for the variable **snapshotindex** in the **config.yaml** file. Otherwise, by default the most recent snapshot is used to analyse the video.

The results are stored in hd5 file in the same directory where the video resides. 

In [25]:
print("Start Analyzing my video(s)!")
scorername = deeplabcut.analyze_videos(path_config_file, 
                                       videofile_path, 
                                       shuffle=shuffle, 
                                       videotype=VideoType, 
                                       c_engine=False)

Start Analyzing my video(s)!
Using snapshot-10000 for model /content/maDLC_MM_hands_trained/maDLC_MM_hands-Brice-2021-07-08/dlc-models/iteration-0/maDLC_MM_handsJul8-trainset95shuffle1
Initializing ResNet
Activating extracting of PAFs
Analyzing all the videos in the directory...
Starting to analyze %  /content/maDLC_MM_hands_trained/Inference_videos/Pilot05-2_CUT.mp4
/content/maDLC_MM_hands_trained/Inference_videos  already exists!
Video already analyzed! /content/maDLC_MM_hands_trained/Inference_videos/Pilot05-2_CUTDLC_resnet50_maDLC_MM_handsJul8shuffle1_10000.h5
Starting to analyze %  /content/maDLC_MM_hands_trained/Inference_videos/Pilot03-2.mp4
/content/maDLC_MM_hands_trained/Inference_videos  already exists!
Video already analyzed! /content/maDLC_MM_hands_trained/Inference_videos/Pilot03-2DLC_resnet50_maDLC_MM_handsJul8shuffle1_10000.h5
Starting to analyze %  /content/maDLC_MM_hands_trained/Inference_videos/Pilot06-2_CUT.mp4
/content/maDLC_MM_hands_trained/Inference_videos  alread

## The steps below work on a single video at a time.
- Here you can create a video to check the pose estimation detection quality! If this looks good, proceed to tracklet conversions with the interactive GUI (ouside of COLAB for now), or if you know your optimal parameters, you can automate this and run the additional steps shown in a few cells down.

In [43]:
##### PROTIP: #####
## look at the output video; if the pose estimation (i.e. key points)
##  don't look good, don't proceed with tracking - add more data to your training set and re-train!

#let's check a specific video (PLEASE EDIT VIDEO PATH):
Specific_videofile = '/content/maDLC_MM_hands_trained/Inference_videos/Video_Pers2_CUT.mp4'

# from deeplabcut.utils import auxiliaryfunctions
# cfg = auxiliaryfunctions.read_config(path_config_file)
# #cfg=95
# scorername, DLCscorerlegacy = auxiliaryfunctions.GetScorerName(cfg, shuffle, trainFraction=0)
# print("scorername is: "+scorername)
# print(cfg)

deeplabcut.create_video_with_all_detections(path_config_file, [Specific_videofile], scorername)

#Again, if this does not look perfect, do not proceed! Retrain with more diverse data.



#from deeplabcut.utils import auxiliaryfunctions
#scorername, DLCscorerlegacy = auxiliaryfunctions.GetScorerName(path_config_file, shuffle, trainFraction=0)
#print("scorename is: "+scorername)

#deeplabcut.create_video_with_all_detections(path_config_file, [Specific_videofile], scorername)

#Again, if this does not look perfect, do not proceed! Retrain with more diverse data.

  7%|▋         | 14/194 [00:00<00:01, 134.93it/s]

Creating labeled video for  Video_Pers2_CUT


100%|██████████| 194/194 [00:01<00:00, 152.17it/s]


## Convert Detections to Tracklets:

- The idea is that you test and adapt hyperparameters for tracking outside of COLAB. Once you have good parameters, this can be automated on future videos. Shown here!

- I.e., instead of always doing an interactive parameter setting step, you can simply convert tracklets to .h5 files using these parameters (see GitHub for more info).

In [44]:
#assemble tracklets:
#read the docs: which tracker to test out (you can run this many times to try multiple):
tracktype= 'box' #box, skeleton, ellipse

deeplabcut.convert_detections2tracklets(path_config_file, Specific_videofile, videotype=VideoType,
                                                    shuffle=shuffle, track_method=tracktype, overwrite=True)

32it [00:00, 310.74it/s]

Using snapshot-10000 for model /content/maDLC_MM_hands_trained/maDLC_MM_hands-Brice-2021-07-08/dlc-models/iteration-0/maDLC_MM_handsJul8-trainset95shuffle1
Processing...  /content/maDLC_MM_hands_trained/Inference_videos/Video_Pers2_CUT.mp4
/content/maDLC_MM_hands_trained/Inference_videos  already exists!
Analyzing /content/maDLC_MM_hands_trained/Inference_videos/Video_Pers2_CUTDLC_resnet50_maDLC_MM_handsJul8shuffle1_10000.h5


194it [00:00, 290.83it/s]

The tracklets were created. Now you can 'refine_tracklets'.


## Now you should manually verify the tracks and correct them if needed! [currently only working outside of COLAB]

In [ ]:
''' here is the code you would need:
os.environ["DLClight"]="False"
import deeplabcut

#ATTENTION:
picklefile = '/...._10000_bx.pickle' #(see your video folder for path i.e. right click and say copy path!!!)
vid ='/yourVIDEO.mp4'
#if you want occlusions filled in, tell us how many frames to fill in, i.e. if there is a gap in data:
framestofill = 0. #note, put "0" if you want ALL gaps filled!

%matplotlib inline

from deeplabcut import refine_tracklets
TrackletManager, TrackletVisualizer = refine_tracklets(path_config_file, 
                                                          picklefile, 
                                                          Specific_videofile, 
                                                          min_swap_frac=0,
                                                          min_tracklet_frac=0, 
                                                          max_gap=framestofill)
'''

## Let's assume you have great tracking parameters, and you want to analyze a full set of videos:

In [45]:
#^^^^^^^^^You do NOT neeed to run if you hit "save" in the GUI ^^^^^^^^^^
#this is just if you want to run the same parameters over a set of videos!

# You need to point to your pickle file, please "copy path" from the folder to the left (right click, copy path)
picklefile = '/content/maDLC_MM_hands_trained/Inference_videos/Video_Pers2_CUTDLC_resnet50_maDLC_MM_handsJul8shuffle1_10000_bx.pickle' #(see your video folder for path i.e. right click and say copy path!!!)
vid ='/content/maDLC_MM_hands_trained/Inference_videos/Video_Pers2_CUT.mp4'

deeplabcut.convert_raw_tracks_to_h5(path_config_file, picklefile)
deeplabcut.filterpredictions(path_config_file, 
                                 videofile_path, 
                                 videotype=VideoType, 
                                 track_method = tracktype)

100%|██████████| 2/2 [00:00<00:00, 547.45it/s]


Analyzing all the videos in the directory...
Filtering with median model /content/maDLC_MM_hands_trained/Inference_videos/Pilot05-2_CUT.mp4
Data from Pilot05-2_CUT were already filtered. Skipping...
Filtering with median model /content/maDLC_MM_hands_trained/Inference_videos/Pilot03-2.mp4
Data from Pilot03-2 were already filtered. Skipping...
Filtering with median model /content/maDLC_MM_hands_trained/Inference_videos/Pilot06-2_CUT.mp4
Data from Pilot06-2_CUT were already filtered. Skipping...
Filtering with median model /content/maDLC_MM_hands_trained/Inference_videos/Video_Pers2_CUT.mp4
Saving filtered csv poses!


## Create plots of your trajectories:

In [46]:
deeplabcut.plot_trajectories(path_config_file,videofile_path, videotype=VideoType, track_method=tracktype)

Analyzing all the videos in the directory...
Loading  /content/maDLC_MM_hands_trained/Inference_videos/Pilot03-2.mp4 and data.
/content/maDLC_MM_hands_trained/Inference_videos/plot-poses/Pilot03-2  already exists!
Loading  /content/maDLC_MM_hands_trained/Inference_videos/Pilot05-2_CUT.mp4 and data.
/content/maDLC_MM_hands_trained/Inference_videos/plot-poses/Pilot05-2_CUT  already exists!
Loading  /content/maDLC_MM_hands_trained/Inference_videos/Video_Pers2_CUT.mp4 and data.
Loading  /content/maDLC_MM_hands_trained/Inference_videos/Pilot06-2_CUT.mp4 and data.
/content/maDLC_MM_hands_trained/Inference_videos/plot-poses/Pilot06-2_CUT  already exists!
Plots created! Please check the directory "plot-poses" within the video directory


Now you can look at the plot-poses file and check the "plot-likelihood.png" might want to change the "p-cutoff" in the config.yaml file so that you have only high confidnece points plotted in the video. i.e. ~0.8 or 0.9. The current default is 0.4. 

## Create labeled video:
This funtion is for visualiztion purpose and can be used to create a video in .mp4 format with labels predicted by the network. This video is saved in the same directory where the original video resides. 

In [47]:
deeplabcut.create_labeled_video(path_config_file,
                                videofile_path, 
                                shuffle=shuffle, 
                                draw_skeleton=True, 
                                videotype=VideoType, 
                                save_frames=False,
                                filtered=True, 
                                track_method = tracktype)

Analyzing all the videos in the directory...
/content/maDLC_MM_hands_trained/Inference_videos  already exists!
/content/maDLC_MM_hands_trained/Inference_videos  already exists!
Starting to process video: /content/maDLC_MM_hands_trained/Inference_videos/Pilot03-2.mp4
Starting to process video: /content/maDLC_MM_hands_trained/Inference_videos/Video_Pers2_CUT.mp4
Loading /content/maDLC_MM_hands_trained/Inference_videos/Video_Pers2_CUT.mp4 and data.
Loading /content/maDLC_MM_hands_trained/Inference_videos/Pilot03-2.mp4 and data.
Labeled video already created. Skipping...
Duration of video [s]: 6.47, recorded with 30.0 fps!
Overall # of frames: 194 with cropped frame dimensions: 852 480
/content/maDLC_MM_hands_trained/Inference_videos  already exists!
Generating frames and creating video.
Starting to process video: /content/maDLC_MM_hands_trained/Inference_videos/Pilot05-2_CUT.mp4
Loading /content/maDLC_MM_hands_trained/Inference_videos/Pilot05-2_CUT.mp4 and data.


  0%|          | 0/194 [00:00<?, ?it/s]

Labeled video already created. Skipping...
/content/maDLC_MM_hands_trained/Inference_videos  already exists!
Starting to process video: /content/maDLC_MM_hands_trained/Inference_videos/Pilot06-2_CUT.mp4
Loading /content/maDLC_MM_hands_trained/Inference_videos/Pilot06-2_CUT.mp4 and data.
Labeled video already created. Skipping...


100%|██████████| 194/194 [00:01<00:00, 104.35it/s]


## (OLD) Start Analyzing videos: 
This function analyzes the new video. The user can choose the best model from the evaluation results and specify the correct snapshot index for the variable **snapshotindex** in the **config.yaml** file. Otherwise, by default the most recent snapshot is used to analyse the video.

The results are stored in hd5 file in the same directory where the video resides. 

In [ ]:
#create a path variable that links to the config file:
videofile_path = ['/content/Pilot5/Pilot05-2_CUT.mp4']
print(videofile_path)

In [ ]:
deeplabcut.analyze_videos(path_config_file, videofile_path, save_as_csv=True)

## (OLD) Plot the trajectories of the analyzed videos:
This function plots the trajectories of all the body parts across the entire video. Each body part is identified by a unique color.

In [ ]:
deeplabcut.plot_trajectories(path_config_file,videofile_path)
#, videotype=VideoType

Now you can look at the plot-poses file and check the "plot-likelihood.png" might want to change the "p-cutoff" in the config.yaml file so that you have only high confidnece points plotted in the video. i.e. ~0.8 or 0.9. The current default is 0.4. 

## (OLD) Create labeled video:
This funtion is for visualiztion purpose and can be used to create a video in .mp4 format with labels predicted by the network. This video is saved in the same directory where the original video resides. 

In [ ]:
deeplabcut.create_labeled_video(path_config_file,videofile_path)
#, videotype=VideoType

In [48]:
!zip -r /content/file.zip /content/maDLC_MM_hands_trained/maDLC_MM_hands-Brice-2021-07-08

  adding: content/maDLC_MM_hands_trained/maDLC_MM_hands-Brice-2021-07-08/ (stored 0%)
  adding: content/maDLC_MM_hands_trained/maDLC_MM_hands-Brice-2021-07-08/videos/ (stored 0%)
  adding: content/maDLC_MM_hands_trained/maDLC_MM_hands-Brice-2021-07-08/videos/Pil05.mp4 (deflated 0%)
  adding: content/maDLC_MM_hands_trained/maDLC_MM_hands-Brice-2021-07-08/videos/Pil06.mp4 (deflated 0%)
  adding: content/maDLC_MM_hands_trained/maDLC_MM_hands-Brice-2021-07-08/videos/VPers1.mp4 (deflated 0%)
  adding: content/maDLC_MM_hands_trained/maDLC_MM_hands-Brice-2021-07-08/videos/Pil03.mp4 (deflated 1%)
  adding: content/maDLC_MM_hands_trained/maDLC_MM_hands-Brice-2021-07-08/videos/Pil02.mp4 (deflated 1%)
  adding: content/maDLC_MM_hands_trained/maDLC_MM_hands-Brice-2021-07-08/videos/Pil01.mp4 (deflated 0%)
  adding: content/maDLC_MM_hands_trained/maDLC_MM_hands-Brice-2021-07-08/labeled-data/ (stored 0%)
  adding: content/maDLC_MM_hands_trained/maDLC_MM_hands-Brice-2021-07-08/labeled-data/Pil05/ (sto

Quantification

In [50]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# set the plots to display in the Jupyter notebook
%matplotlib inline

# change plotting colors per client request
plt.style.use('ggplot')

# Increase default figure and font sizes for easier viewing.
plt.rcParams['figure.figsize'] = (8, 6)
plt.rcParams['font.size'] = 14

In [71]:
video = pd.read_csv('/content/maDLC_MM_hands_trained/Inference_videos/Video_Pers2_CUTDLC_resnet50_maDLC_MM_handsJul8shuffle1_10000_bx_filtered.csv')

In [74]:
df.columns = [
'LThumbstartX',
'LThumbstartY',
'LThumbstartLike',
'LThumbendX',
'LThumbendY',
'LThumbendLike',
'LIndexstartX',
'LIndexstartY',
'LIndexstartLike',
'LIndexendX',
'LIndexendY',
'LIndexendLike',
'LMiddlestartX',
'LMiddlestartY',
'LMiddlestartLike',
'LMiddleendX',
'LMiddleendY',
'LMiddleendLike',
'LRingstartX',
'LRingstartY',
'LRingstartLike',
'LRingendX',
'LRingendY',
'LRingendLike',
'LLittlestartX',
'LLittlestartY',
'LLittlestartLike',
'LLittleendX',
'LLittleendY',
'LLittleendLike',
'RThumbstartX',
'RThumbstartY',
'RThumbstartLike',
'RThumbendX',
'RThumbendY',
'RThumbendLike',
'RIndexstartX',
'RIndexstartY',
'RIndexstartLike',
'RIndexendX',
'RIndexendY',
'RIndexendLike',
'RMiddlestartX',
'RMiddlestartY',
'RMiddlestartLike',
'RMiddleendX',
'RMiddleendY',
'RMiddleendLike',
'RRingstartX',
'RRingstartY',
'RRingstartLike',
'RRingendX',
'RRingendY',
'RRingendLike',
'RLittlestartX',
'RLittlestartY',
'RLittlestartLike',
'RLittleendX',
'RLittleendY',
'RLittleendLike'
]

# video = video.iloc[1:]
# video.drop(video.index[0])
# video.drop(video.index[1])
# video.drop(video.index[2])
#video.drop(video.columns[[1]], axis=1)



In [75]:
video.head(3)

,scorer,DLC_resnet50_maDLC_MM_handsJul8shuffle1_10000,DLC_resnet50_maDLC_MM_handsJul8shuffle1_10000.1,DLC_resnet50_maDLC_MM_handsJul8shuffle1_10000.2,DLC_resnet50_maDLC_MM_handsJul8shuffle1_10000.3,DLC_resnet50_maDLC_MM_handsJul8shuffle1_10000.4,DLC_resnet50_maDLC_MM_handsJul8shuffle1_10000.5,DLC_resnet50_maDLC_MM_handsJul8shuffle1_10000.6,DLC_resnet50_maDLC_MM_handsJul8shuffle1_10000.7,DLC_resnet50_maDLC_MM_handsJul8shuffle1_10000.8,DLC_resnet50_maDLC_MM_handsJul8shuffle1_10000.9,DLC_resnet50_maDLC_MM_handsJul8shuffle1_10000.10,DLC_resnet50_maDLC_MM_handsJul8shuffle1_10000.11,DLC_resnet50_maDLC_MM_handsJul8shuffle1_10000.12,DLC_resnet50_maDLC_MM_handsJul8shuffle1_10000.13,DLC_resnet50_maDLC_MM_handsJul8shuffle1_10000.14,DLC_resnet50_maDLC_MM_handsJul8shuffle1_10000.15,DLC_resnet50_maDLC_MM_handsJul8shuffle1_10000.16,DLC_resnet50_maDLC_MM_handsJul8shuffle1_10000.17,DLC_resnet50_maDLC_MM_handsJul8shuffle1_10000.18,DLC_resnet50_maDLC_MM_handsJul8shuffle1_10000.19,DLC_resnet50_maDLC_MM_handsJul8shuffle1_10000.20,DLC_resnet50_maDLC_MM_handsJul8shuffle1_10000.21,DLC_resnet50_maDLC_MM_handsJul8shuffle1_10000.22,DLC_resnet50_maDLC_MM_handsJul8shuffle1_10000.23,DLC_resnet50_maDLC_MM_handsJul8shuffle1_10000.24,DLC_resnet50_maDLC_MM_handsJul8shuffle1_10000.25,DLC_resnet50_maDLC_MM_handsJul8shuffle1_10000.26,DLC_resnet50_maDLC_MM_handsJul8shuffle1_10000.27,DLC_resnet50_maDLC_MM_handsJul8shuffle1_10000.28,DLC_resnet50_maDLC_MM_handsJul8shuffle1_10000.29,DLC_resnet50_maDLC_MM_handsJul8shuffle1_10000.30,DLC_resnet50_maDLC_MM_handsJul8shuffle1_10000.31,DLC_resnet50_maDLC_MM_handsJul8shuffle1_10000.32,DLC_resnet50_maDLC_MM_handsJul8shuffle1_10000.33,DLC_resnet50_maDLC_MM_handsJul8shuffle1_10000.34,DLC_resnet50_maDLC_MM_handsJul8shuffle1_10000.35,DLC_resnet50_maDLC_MM_handsJul8shuffle1_10000.36,DLC_resnet50_maDLC_MM_handsJul8shuffle1_10000.37,DLC_resnet50_maDLC_MM_handsJul8shuffle1_10000.38,DLC_resnet50_maDLC_MM_handsJul8shuffle1_10000.39,DLC_resnet50_maDLC_MM_handsJul8shuffle1_10000.40,DLC_resnet50_maDLC_MM_handsJul8shuffle1_10000.41,DLC_resnet50_maDLC_MM_handsJul8shuffle1_10000.42,DLC_resnet50_maDLC_MM_handsJul8shuffle1_10000.43,DLC_resnet50_maDLC_MM_handsJul8shuffle1_10000.44,DLC_resnet50_maDLC_MM_handsJul8shuffle1_10000.45,DLC_resnet50_maDLC_MM_handsJul8shuffle1_10000.46,DLC_resnet50_maDLC_MM_handsJul8shuffle1_10000.47,DLC_resnet50_maDLC_MM_handsJul8shuffle1_10000.48,DLC_resnet50_maDLC_MM_handsJul8shuffle1_10000.49,DLC_resnet50_maDLC_MM_handsJul8shuffle1_10000.50,DLC_resnet50_maDLC_MM_handsJul8shuffle1_10000.51,DLC_resnet50_maDLC_MM_handsJul8shuffle1_10000.52,DLC_resnet50_maDLC_MM_handsJul8shuffle1_10000.53,DLC_resnet50_maDLC_MM_handsJul8shuffle1_10000.54,DLC_resnet50_maDLC_MM_handsJul8shuffle1_10000.55,DLC_resnet50_maDLC_MM_handsJul8shuffle1_10000.56,DLC_resnet50_maDLC_MM_handsJul8shuffle1_10000.57,DLC_resnet50_maDLC_MM_handsJul8shuffle1_10000.58,DLC_resnet50_maDLC_MM_handsJul8shuffle1_10000.59
1,bodyparts,thumbStart,thumbStart,thumbStart,thumbEnd,thumbEnd,thumbEnd,indexStart,indexStart,indexStart,indexEnd,indexEnd,indexEnd,middleStart,middleStart,middleStart,middleEnd,middleEnd,middleEnd,ringStart,ringStart,ringStart,ringEnd,ringEnd,ringEnd,littleStart,littleStart,littleStart,littleEnd,littleEnd,littleEnd,thumbStart,thumbStart,thumbStart,thumbEnd,thumbEnd,thumbEnd,indexStart,indexStart,indexStart,indexEnd,indexEnd,indexEnd,middleStart,middleStart,middleStart,middleEnd,middleEnd,middleEnd,ringStart,ringStart,ringStart,ringEnd,ringEnd,ringEnd,littleStart,littleStart,littleStart,littleEnd,littleEnd,littleEnd
2,coords,x,y,likelihood,x,y,likelihood,x,y,likelihood,x,y,likelihood,x,y,likelihood,x,y,likelihood,x,y,likelihood,x,y,likelihood,x,y,likelihood,x,y,likelihood,x,y,likelihood,x,y,likelihood,x,y,likelihood,x,y,likelihood,x,y,likelihood,x,y,likelihood,x,y,likelihood,x,y,likelihood,x,y,likelihood,x,y,likelihood
3,0,351.25,185.375,1.0,404.75,240.25,1.0,311.25,218.5,1.0,331.75,339.5,1.0,273.25,223.375,1.0,265.5,355.25,1.0,230.125,222.375,1.0,